In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import os, sys
import json

from run_search import find_similar_vectors

Using this notebook to develop the prompt for Llama3.1. 

Requires that the DB already made.

In [2]:
vector_query = '''
what nodes and edges should be used for a knowledge graph that will be used for drug repurposing or target discovery?
'''
path_config = '/users/anglup/GitHub/pubmed-rag/config.yaml'
path_to_repo = '/users/anglup/github/pubmed-rag'

In [3]:
results = find_similar_vectors(
    path_config,
    vector_query,
)

Path to config file: /users/anglup/GitHub/pubmed-rag/config.yaml
Loading config params ... 
Configuration: {'pmid file path': 'notebooks/data/test_pmid_list.csv', 'biocjson output path': 'biocjson', 'max_tokens': 300, 'transformer_model': 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', 'db name': '/users/anglup/github/pubmed-rag/demo.db', 'tsne': 'tsne.png', 'n_results': 10, 'output_dimensions': 768}
Accessing /users/anglup/github/pubmed-rag/demo.db
Loading model microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext from HuggingFace


/Users/anglup/Library/Caches/pypoetry/virtualenvs/pubmed-rag-iB5Z1S4t-py3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embedding query: 
what nodes and edges should be used for a knowledge graph that will be used for drug repurposing or target discovery?

Searching against vector database


In [5]:
for x in (results):
    print(json.dumps(x, indent=4))

{
    "id": 711,
    "distance": 0.9874027371406555,
    "entity": {
        "section": "intro",
        "pmid": 33175089,
        "text": "1 Introduction . In the absence of effective medications for COVID-19, there is an urgent need to identify drugs that can combat this ongoing pandemic . This task can be accomplished most rapidly by repurposing the existing medications . Biomedical knowledge graphs such as Reasoning Over Biomedical Objects linked in Knowledge Oriented Pathways (ROBOKOP) provide an efficient way to identify potential candidate drugs by making inferences on the relationships between knowledge graph nodes . We have merged the ROBOKOP knowledge graph with the new supply of COVID-19-related information from recent publications and other knowledge sources to form COVID-KOP."
    }
}
{
    "id": 1011,
    "distance": 0.9858823418617249,
    "entity": {
        "section": "discuss",
        "pmid": 36258252,
        "text": "This pattern suggests a focus on label function 